<a href="https://colab.research.google.com/github/makkyo-git/python-collection/blob/mabo/DCGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

まず、Google Driveのマウントをします

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


GANした画像を入れるディレクトリを作らないといけないので、ターミナルをインストールします。

In [ ]:
%mkdir GAN

**<font size = 5.5 >ライブラリのインポート**

GANで、手書き文字を作るには、以下のライブラリをインポートする必要があります。

In [ ]:
# -*- coding: utf-8 -*-

import os
import random
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import matplotlib.pyplot as plt
import copy
from collections import OrderedDict
from torch.utils.data import dataset

それでは、与えられた画像(データセット)を元に、GAN(敵対的ネットワーク生成)をさせて、精度を上げていき、手書き文字のように文字を作成するためのコードを書いていきます。

In [ ]:
# Initial_setting
workers = 5
batch_size=32
nz = 100
nch_g = 64
nch_d = 64
n_epoch = 10000
lr = 0.001
beta1 = 0.5
outf = './result_lsgan'
display_interval = 100
save_fake_image_interval = 1500
plt.rcParams['figure.figsize'] = 10, 6

try:
    os.makedirs(outf, exist_ok=True)
except OSError as error:
    print(error)
    pass

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)


def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
        m.bias.data.fill_(0)
    elif classname.find('Linear') != -1:
        m.weight.data.normal_(0.0, 0.02)
        m.bias.data.fill_(0)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)


class Generator(nn.Module):
    def __init__(self, nz=100, nch_g=64, nch=3):
        super(Generator, self).__init__()
        self.layers = nn.ModuleDict({
            'layer0': nn.Sequential(
                nn.ConvTranspose2d(nz, nch_g * 8, 4, 1, 0),
                nn.BatchNorm2d(nch_g * 8),
                nn.ReLU()
            ),  # (100, 1, 1) -> (512, 4, 4)
            'layer1': nn.Sequential(
                nn.ConvTranspose2d(nch_g * 8, nch_g * 4, 4, 2, 1),
                nn.BatchNorm2d(nch_g * 4),
                nn.ReLU()
            ),  # (512, 4, 4) -> (256, 8, 8)
            'layer2': nn.Sequential(
                nn.ConvTranspose2d(nch_g * 4, nch_g * 2, 4, 2, 1),
                nn.BatchNorm2d(nch_g * 2),
                nn.ReLU()
            ),  # (256, 8, 8) -> (128, 16, 16)

            'layer3': nn.Sequential(
                nn.ConvTranspose2d(nch_g * 2, nch_g, 4, 2, 1),
                nn.BatchNorm2d(nch_g),
                nn.ReLU()
            ),  # (128, 16, 16) -> (64, 32, 32)
            'layer4': nn.Sequential(
                nn.ConvTranspose2d(nch_g, nch, 4, 2, 1),
                nn.Tanh()    
            )   # (64, 32, 32) -> (3, 64, 64)
        })

    def forward(self, z):
        for layer in self.layers.values():
            z = layer(z)
        return z

class Discriminator(nn.Module):
    def __init__(self, nch=3, nch_d=64):
        super(Discriminator, self).__init__()
        self.layers = nn.ModuleDict({
            'layer0': nn.Sequential(
                nn.Conv2d(nch, nch_d, 4, 2, 1),
                nn.BatchNorm2d(nch_d),
                nn.LeakyReLU(negative_slope=0.2)
            ), # (3, 64, 64) -> (64, 32, 32)
            'layer1': nn.Sequential(
                nn.Conv2d(nch_d, nch_d * 2, 4, 2, 1),
                nn.BatchNorm2d(nch_d*2),
                nn.LeakyReLU(negative_slope=0.2)
            ), # (64, 32, 32) -> (128, 16, 16)
            'layer2': nn.Sequential(
                nn.Conv2d(nch_d * 2, nch_d * 4, 4, 2, 1),
                nn.BatchNorm2d(nch_d*4),
                nn.LeakyReLU(negative_slope=0.2)
            ), # (128, 16, 16) -> (256, 8, 8)
            'layer3': nn.Sequential(
                nn.Conv2d(nch_d * 4, nch_d * 8, 4, 2, 1),
                nn.BatchNorm2d(nch_d*8),
                nn.LeakyReLU(negative_slope=0.2)
            ), # (256, 8, 8) -> (512, 4, 4)
            'layer4':nn.Sequential( nn.Conv2d(nch_d * 8, 1, 4, 1, 0)
            # (512, 4, 4) -> (1, 1, 1)
            # 勾配消失を防ぐためにSigmoidは使わない
            )
        })

    def forward(self, x):
        for layer in self.layers.values():
            x = layer(x)
            x = x.squeeze()

        return x


def main():
 #パスには入っている画像データセットの一個上の階層を
 #例:もし  your_home/Face_Datasets/Japanese/000.jpgのような階層になっていれば
 #root = your_home/Face_Datasetsとするとする
    dataset = dset.ImageFolder(root='/content/drive/MyDrive/dataset/',
                               transform=transforms.Compose([
                               transforms.RandomResizedCrop(64, scale=(1.0, 1.0), ratio=(1., 1.)),
                               transforms.ColorJitter(brightness=0.05, contrast=0.05, saturation=0.05, hue=0.05),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                          ]))
    
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=int(workers))

    #GPUがあるならGPUデバイスを作動
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    #Generatorを定義
    netG = Generator(nz=nz, nch_g=nch_g).to(device)
    #ネットワークパラメータをランダムに初期化
    netG.apply(weights_init)

    #Discriminatorを定義
    netD = Discriminator(nch_d=nch_d).to(device)

    #ネットワークパラメータをランダムに初期化
    netD.apply(weights_init)

    #損失関数を二乗誤差に設定
    criterion = nn.MSELoss()

    #それぞれのパラメータ更新用のoptimizerを定義
    optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999), weight_decay=1e-5)
    optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999), weight_decay=1e-5)

    Loss_D_list, Loss_G_list = [],[]

    fixed_noise = torch.randn(batch_size, nz, 1, 1, device=device) # save_fake_image用ノイズ（固定）
    for epoch in range(n_epoch):

        for itr, data in enumerate(dataloader):

            real_image = data[0].to(device) #本物画像

            sample_size = real_image.size(0) #画像枚数
            noise = torch.randn(sample_size, nz, 1, 1, device=device) # 入力ベクトル生成（正規分布ノイズ）
            real_target = torch.full((sample_size,), random.uniform(1, 1), device=device) #　本物ラベル
            fake_target = torch.full((sample_size,), random.uniform(0, 0), device=device) #  偽物ラベル


            #Generator→Discriminatorの順にパラメータ更新

            #----------  Update Gnerator  ----------
            netG.zero_grad()
            fake_image = netG(noise) # Generatorから得られた偽画像
            output = netD(fake_image)  # 更新した Discriminatorで、偽物画像を判定
            errG = criterion(output,real_target) # 偽物画像の判定結果と本物ラベルとの二乗誤差
            errG.backward(retain_graph = True) # 誤差逆伝播
            D_G_z2 = output.mean().item() # outputの平均　D_G_z2 を計算 (後でログ出力に使用)

            optimizerG.step() # Generatorのパラメータ更新


            #------- Update Discriminator ------------
            netD.zero_grad() # 勾配の初期化
            fake_image = netG(noise) # Generatorから得られた偽画像
            output = netD(real_image) # Discriminatorが行った、本物画像の判定結果
            errD_real = criterion(output,real_target) # 本物画像の判定結果と本物ラベルとの二乗誤差
            D_x = output.mean().item() # outputの平均　D_x を計算 (後にログ出力に使用)

            output = netD(fake_image.detach()) # Discriminatorが行った、偽物画像の判定結果

            errD_fake = criterion(output,fake_target) # 偽物画像の判定結果と偽物画像との二乗誤差二乗誤差
            D_G_z1 = output.mean().item() # outputの平均　D_G_z1 を計算 (後でログ出力に使用)

            errD = errD_real + errD_fake # Discriminator 全体の損失
            errD.backward(retain_graph = True) # 誤差逆伝播
            optimizerD.step() # Discriminatorのパラメーターを更新


            # 定期的に損失を表示
            if itr % 5 == 0:
                print('[{}/{}][{}/{}] Loss_D: {:.3f} Loss_G: {:.3f} D(x): {:.3f} D(G(z)): {:.3f}/{:.3f}'
                      .format(epoch + 1, n_epoch,
                              itr + 1, len(dataloader),
                              errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))
            
                Loss_D_list.append(errD.item())
                Loss_G_list.append(errG.item())

            # 定期的に画像を保存
            if (itr + 1) % 16 == 0:
                
                fake_image = netG(fixed_noise)
                # 出力した画像を保存するため、ファイルというメニューの中に、'GAN'という空のディレクトリを作成し、出力できた画像を入れていく。
                vutils.save_image(fake_image.detach(), './GAN/{:03d}random_{:03d}.png'.format(itr,epoch + 1),
                                  normalize=True, nrow=8)
           
    # plot graph
    plt.figure()
    plt.plot(range(len(Loss_D_list)), Loss_D_list, color='blue', linestyle='-', label='Loss_D')
    plt.plot(range(len(Loss_G_list)), Loss_G_list, color='red', linestyle='-', label='Loss_G')
    plt.legend()
    plt.xlabel('iter (*100)')
    plt.ylabel('loss')
    plt.title('Loss_D and Loss_G')
    plt.grid()
    plt.savefig('Loss_graph.png')

if __name__ == '__main__':
    main()


                
          

        

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[1/10000][1/32] Loss_D: 174.158 Loss_G: 4.790 D(x): 0.572 D(G(z)): 12.832/-1.020
[1/10000][6/32] Loss_D: 162.137 Loss_G: 244.931 D(x): 4.472 D(G(z)): -11.997/16.496
[1/10000][11/32] Loss_D: 2.808 Loss_G: 12.449 D(x): 0.735 D(G(z)): 1.138/-2.443
[1/10000][16/32] Loss_D: 12.685 Loss_G: 91.510 D(x): 0.307 D(G(z)): 3.323/-8.524
[1/10000][21/32] Loss_D: 4.931 Loss_G: 0.366 D(x): 0.483 D(G(z)): 1.990/1.449
[1/10000][26/32] Loss_D: 0.607 Loss_G: 2.186 D(x): 0.590 D(G(z)): -0.108/-0.408
[1/10000][31/32] Loss_D: 0.284 Loss_G: 1.165 D(x): 1.079 D(G(z)): 0.249/-0.033
[2/10000][1/32] Loss_D: 0.573 Loss_G: 2.256 D(x): 0.436 D(G(z)): -0.115/-0.478
[2/10000][6/32] Loss_D: 0.228 Loss_G: 1.965 D(x): 1.004 D(G(z)): 0.204/-0.380
[2/10000][11/32] Loss_D: 0.398 Loss_G: 8.690 D(x): 0.705 D(G(z)): 0.213/-1.936
[2/10000][16/32] Loss_D: 10.594 Loss_G: 17.354 D(x): 2.896 D(G(z)): 2.357/5.043
[2/10000][21/32] Loss_D: 1.518 Loss_G: 6.698 D(x): 0.695 D(G(z)): 0.887/-1.546
[2/10000][26/32] Loss_D: 28.452 Loss_G: 2.